## 02 - Cleaning Traffy Data (Bangkok 2024+)

Target of this notebook

- ทำความสะอาดข้อมูลดิบจาก Traffy
- เลือกเฉพาะเคสที่อยู่ใน "กรุงเทพมหานคร"
- เลือกช่วงปี >= 2024
- สร้างคอลัมน์มาตรฐาน เช่น province_clean, std_type, lat/lng
- สุ่มเหลือ 150,000 rows สำหรับนำไปทำ ML (`random_state=42`)
- เซฟเป็นไฟล์ processed + sample สำหรับ GitHub

In [58]:
import pandas as pd
import numpy as np

In [59]:
df = pd.read_csv('../data/bangkok_traffy.csv')
df.shape

(787026, 16)

In [60]:
df.head(5)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


## Convert timestamp & last_activity to datetime

- เก็บปีของ `timestamp` ไว้ในคอลัมน์ `year`


In [61]:
df["timestamp"] = pd.to_datetime(df["timestamp"], format="mixed", errors="coerce")
df["last_activity"] = pd.to_datetime(df["last_activity"], format="mixed", errors="coerce")

df["year"] = df["timestamp"].dt.year

print(df["timestamp"].min())
print(df["timestamp"].max())
df["year"].value_counts().sort_index(ascending=False)


2021-09-03 12:51:09.453003+00:00
2025-01-16 02:53:34.290375+00:00


year
2025     12719
2024    307931
2023    276399
2022    189965
2021        12
Name: count, dtype: int64

## Province cleaning & filter to Bangkok

- สร้างคอลัมน์ `province_clean` เพื่อรวมค่าที่เขียนต่างกันให้เหลือกลุ่มเดียว
- เลือกเฉพาะ record ที่อยู่ในกรุงเทพฯ เท่านั้น


In [62]:
def clean_province(x: str) -> str:
    if pd.isna(x):
        return ""
    s = str(x)
    # ตัดคำว่า "จังหวัด" และช่องว่าง / สัญลักษณ์พื้นฐาน
    s = s.replace("จังหวัด", "")
    s = s.replace(" ", "")
    s = s.replace("ฯ", "")
    s = s.replace(".", "")
    return s.lower()

df["province_clean"] = df["province"].apply(clean_province)

df["province_clean"].value_counts().head(20)


province_clean
กรุงเทพมหานคร      785654
lac                   293
นนทบุรี               261
สมุทรปราการ           244
                      196
ปทุมธานี              110
สมุทรสาคร              45
นครปฐม                 30
นครราชสีมา             21
borno                  17
ภูเก็ต                 14
เพชรบุรี               13
ราชบุรี                12
ชลบุรี                 10
ฉะเชิงเทรา              7
bangkok                 6
เชียงใหม่               6
sahel                   5
พระนครศรีอยุธยา         5
ลพบุรี                  5
Name: count, dtype: int64

In [63]:
df_bkk = df[df["province_clean"] == "กรุงเทพมหานคร"].copy()
print("After Bangkok filter:", df_bkk.shape)
df_bkk["province_clean"].value_counts()

After Bangkok filter: (785654, 18)


province_clean
กรุงเทพมหานคร    785654
Name: count, dtype: int64

## Clean `type` and create `std_type`

- แปลง string ใน `type` ให้กลายเป็น list ของหมวดปัญหา
- map เข้าหมวดมาตรฐาน `std_type` (เช่น road, flood, lighting, etc)
- ถ้าไม่เข้าเงื่อนไขใดเลยให้เป็น `other`


In [64]:
def parse_type_cell(x):
    if pd.isna(x):
        return ["unknown"]   
    
    s = str(x).strip()
    s = s.strip("{}")
    parts = [p.strip() for p in s.split(",") if p.strip()]
    
    if not parts:
        return ["unknown"]   
    return parts


df_bkk["type_list"] = df_bkk["type"].apply(parse_type_cell)
df_bkk[["type", "type_list"]].sample(10,random_state=42)


,type,type_list
600308,{},[unknown]
126623,{สายไฟ},[สายไฟ]
315309,{},[unknown]
82480,{ความสะอาด},[ความสะอาด]
364858,"{ถนน,ความสะอาด}","[ถนน, ความสะอาด]"
92323,{น้ำท่วม},[น้ำท่วม]
735164,{จราจร},[จราจร]
269253,"{ถนน,สัตว์จรจัด,ร้องเรียน}","[ถนน, สัตว์จรจัด, ร้องเรียน]"
381432,{จราจร},[จราจร]
555775,{กีดขวาง},[กีดขวาง]


In [65]:
type_map = {
    # -----------------
    # ROAD / TRAFFIC / SIGN
    # -----------------
    "ถนน": "road",
    "ทางเท้า": "road",
    "สะพาน": "road",
    "จราจร": "road",
    "ป้าย": "road",          
    "ป้ายจราจร": "road",
    "คมนาคม": "road",
    "การเดินทาง": "road",
    "ผิวจราจร": "road",
    "ฟุตบาท": "road",
    "พื้นผิว": "road",
    "กีดขวาง": "road",      

    # -----------------
    # FLOOD / WATER
    # -----------------
    "น้ำท่วม": "flood",
    "น้ำท่วมขัง": "flood",
    "น้ำรอระบาย": "flood",
    "น้ำ": "flood",          

    # -----------------
    # DRAINAGE
    # -----------------
    "ท่อระบายน้ำ": "drain",
    "ท่อ": "drain",
    "ท่อระบาย": "drain",
    "บ่อพัก": "drain",

    # -----------------
    # LIGHTING / ELECTRIC
    # -----------------
    "แสงสว่าง": "lighting",
    "ไฟฟ้า": "lighting",
    "ไฟ": "lighting",
    "ไฟถนน": "lighting",
    "ไฟทาง": "lighting",
    "เสาไฟ": "lighting",
    "สายไฟ": "lighting",

    # -----------------
    # CLEANING / GARBAGE
    # -----------------
    "ความสะอาด": "cleaning",
    "ขยะ": "cleaning",
    "สิ่งปฏิกูล": "cleaning",
    "คราบสกปรก": "cleaning",

    # -----------------
    # SAFETY / RISK / PM2.5
    # -----------------
    "ความปลอดภัย": "safety",
    "อันตราย": "safety",
    "เสี่ยงอันตราย": "safety",
    "ต้นไม้": "safety",          
    "กิ่งไม้": "safety",
    "PM2.5": "safety",
    "ควัน": "safety",
    "ไฟไหม้": "safety",
    "คนจรจัด": "safety",
    "มั่วสุม": "safety",

    # -----------------
    # NOISE / GENERAL COMPLAINT
    # -----------------
    "เสียงรบกวน": "noise",
    "เสียง": "noise",
    "ร้องเรียน": "noise",        
    "ร้องทุกข์": "noise",
    "งานเลี้ยง": "noise",
    "ลำโพง": "noise",

    # -----------------
    # CANAL / WATERWAY
    # -----------------
    "คลอง": "canal",
    "ลำคลอง": "canal",
    "คูน้ำ": "canal",

    # -----------------
    # ANIMAL
    # -----------------
    "สัตว์จรจัด": "animal",
    "สัตว์": "animal",
    "สุนัข": "animal",
    "หมา": "animal",
    "แมว": "animal",
    "งู": "animal",
    "หนู": "animal",

}

def map_std_type(type_list):

    if "unknown" in type_list:
        return "unknown"
    
    for t in type_list:
        for key, std in type_map.items():
            if key in t:
                return std
    return "other"

df_bkk["std_type"] = df_bkk["type_list"].apply(map_std_type)

df_bkk["std_type"].value_counts()




std_type
road        361403
unknown     115069
lighting     68307
safety       68071
flood        56895
cleaning     45809
noise        44056
canal        13955
animal       10050
other         2039
Name: count, dtype: int64

In [66]:
df_other = df_bkk[df_bkk["std_type"] == "other"]

df_other["type_list"].value_counts().head()


type_list
[สอบถาม]             1252
[เสนอแนะ]             761
[สอบถาม, เสนอแนะ]      18
[เสนอแนะ, สอบถาม]       8
Name: count, dtype: int64

## Clean location fields

- เติมค่า missing ใน `district` / `subdistrict` เป็น `'unknown'`
- แยกพิกัด `coords` ออกเป็น `lng`, `lat` แบบตัวเลข float


In [67]:
df_bkk["district"] = df_bkk["district"].fillna("unknown").str.strip()
df_bkk["subdistrict"] = df_bkk["subdistrict"].fillna("unknown").str.strip()
print("na district",df_bkk['district'].isna().sum())
print("na subdistrict",df_bkk['subdistrict'].isna().sum())

na district 0
na subdistrict 0


In [68]:
def split_coords(x):
    try:
        lon_str, lat_str = str(x).split(",")
        return float(lon_str), float(lat_str)
    except Exception:
        return (np.nan, np.nan)

coords = df_bkk["coords"].apply(split_coords)
df_bkk["lng"] = coords.str[0]
df_bkk["lat"] = coords.str[1]

df_bkk[["coords", "lng", "lat"]].head(3)


,coords,lng,lat
0,"100.53084,13.81865",100.53084,13.81865
1,"100.66709,13.67891",100.66709,13.67891
2,"100.52649,13.72060",100.52649,13.72060


## Filter by year >= 2024 , Comment_length ,sample 150k rows


In [69]:
df_2024plus = df_bkk[df_bkk["year"] >= 2024].copy()

print("After year >= 2024 filter:", df_2024plus.shape)
df_2024plus["year"].value_counts().sort_index(ascending=False)

After year >= 2024 filter: (320056, 22)


year
2025     12679
2024    307377
Name: count, dtype: int64

In [70]:
df_clean = df_2024plus.sample(150000, random_state=42).copy()
print(df_clean.shape)

(150000, 22)


In [71]:
df_clean["comment_length"] = (
    df_clean["comment"]
    .fillna("")
    .astype(str)
    .str.strip()
    .str.len()
)
df_clean["count_reopen"] = df_clean["count_reopen"].fillna(0).astype(int)


In [72]:
df_clean["comment_length"].describe()



count    150000.000000
mean        189.533407
std         277.898954
min           0.000000
25%          44.000000
50%         111.000000
75%         234.000000
max       10620.000000
Name: comment_length, dtype: float64

In [73]:
df_clean.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,...,star,count_reopen,last_activity,year,province_clean,type_list,std_type,lng,lat,comment_length
640580,2024-CVGNCL,{ถนน},"เขตราชเทวี,ฝ่ายเทศกิจ เขตราชเทวี",ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.54204,13.75420",10400 419 ถ. ราชปรารภ แขวงมักกะสัน เขตราชเทวี ...,ถนนพญาไท,ราชเทวี,...,5.0,0,2024-11-03 18:26:10.631498+00:00,2024,กรุงเทพมหานคร,[ถนน],road,100.54204,13.75420,351
747273,2024-ATFNH8,{ความสะอาด},"เขตลาดกระบัง,ฝ่ายรักษาความสะอาดฯ เขตลาดกระบัง",ขยะไม่เก็บ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.73608,13.73114",PPJP+GG6 แขวงคลองสองต้นนุ่น เขตลาดกระบัง กรุงเ...,คลองสองต้นนุ่น,ลาดกระบัง,...,4.0,0,2024-11-30 01:58:13.419161+00:00,2024,กรุงเทพมหานคร,[ความสะอาด],cleaning,100.73608,13.73114,10
755132,ZN3ZZV,"{ถนน,กีดขวาง}","เขตลาดพร้าว,ฝ่ายเทศกิจ เขตลาดพร้าว",ปัญหา: ริมถนนดังกล่าว บริเวณหน้าเซ่เวน อีเลฟเว...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59014,13.80482",RH3R+W2V แขวงลาดพร้าว เขตลาดพร้าว กรุงเทพมหานค...,ลาดพร้าว,ลาดพร้าว,...,NaN,3,2024-12-25 16:53:44.505932+00:00,2024,กรุงเทพมหานคร,"[ถนน, กีดขวาง]",road,100.59014,13.80482,424
567496,2024-ET2E9B,"{ถนน,แสงสว่าง}","เขตคลองสามวา,สำนักการโยธา กทม.,การไฟฟ้านครหลวง...",ไฟส่องสว่าง บริเวณเกาะกลางดับ ถนนสามวา (สามวา ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.72685,13.84308",RPVG+5MX ซอย สามวา 29 แขวง บางชัน เขตคลองสามวา...,บางชัน,คลองสามวา,...,NaN,0,2024-06-18 06:47:53.894049+00:00,2024,กรุงเทพมหานคร,"[ถนน, แสงสว่าง]",road,100.72685,13.84308,54
474259,2024-DRE2UU,{ถนน},"เขตบางพลัด,สำนักการโยธา กทม.,สำนักงานก่อสร้างแ...",ซ่อมถนนตรงนี้ไหนครับเป็นร่องตรงกลางยาว 10 เมตร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.48622,13.77831",354 ถ. จรัญสนิทวงศ์ แขวงบางยี่ขัน เขตบางพลัด ก...,บางยี่ขัน,บางพลัด,...,NaN,0,2024-01-24 08:13:39.004389+00:00,2024,กรุงเทพมหานคร,[ถนน],road,100.48622,13.77831,50


## Save processed data

- เซฟไฟล์หลัก `traffy_clean_150k.csv` ลง `data/` (ไม่ push ขึ้น GitHub)
- เซฟ sample 100 แถวสำหรับ GitHub ที่ `data_samples/02-cleaned_sample.csv`


In [74]:
cols_keep = [
    "ticket_id",
    "std_type",
    "type",
    "type_list",
    "organization",
    "comment",
    "comment_length",
    "photo",
    "photo_after",
    "lng",
    "lat",
    "subdistrict",
    "district",
    "province",
    "province_clean",
    "timestamp",
    "last_activity",
    "state",
    "star",
    "count_reopen",
    "year",
]

df_clean = df_clean[cols_keep]
df_clean.head(2)

,ticket_id,std_type,type,type_list,organization,comment,comment_length,photo,photo_after,lng,...,subdistrict,district,province,province_clean,timestamp,last_activity,state,star,count_reopen,year
640580,2024-CVGNCL,road,{ถนน},[ถนน],"เขตราชเทวี,ฝ่ายเทศกิจ เขตราชเทวี",ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,351,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.54204,...,ถนนพญาไท,ราชเทวี,กรุงเทพมหานคร,กรุงเทพมหานคร,2024-07-31 02:56:49.753428+00:00,2024-11-03 18:26:10.631498+00:00,เสร็จสิ้น,5.0,0,2024
747273,2024-ATFNH8,cleaning,{ความสะอาด},[ความสะอาด],"เขตลาดกระบัง,ฝ่ายรักษาความสะอาดฯ เขตลาดกระบัง",ขยะไม่เก็บ,10,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.73608,...,คลองสองต้นนุ่น,ลาดกระบัง,กรุงเทพมหานคร,กรุงเทพมหานคร,2024-11-28 08:26:36.077122+00:00,2024-11-30 01:58:13.419161+00:00,เสร็จสิ้น,4.0,0,2024


In [75]:
df_clean.shape

(150000, 21)

In [76]:
processed_path = "../data/traffy_clean_150k.csv"
sample_clean_path = "../data_samples/01-cleaned_sample.csv"

df_clean.to_csv(processed_path, index=False)
df_clean.sample(100, random_state=42).to_csv(sample_clean_path, index=False)
